# Ejercicio 1

En esta serie de ejercicios vamos a practicar a renombrar, juntar y combinar DataFrames haciendo uso de las funciones rename(), concat(), merge() y join(). Ejecuta el siguiente código antes de empezar para importar Pandas y definir las URL de los ficheros:

In [ ]:
import pandas as pd

url_customers = "https://raw.githubusercontent.com/cpinuaga/carlospinuagathevalley-IntroPandas2023/main/customers.csv"
url_products = "https://raw.githubusercontent.com/cpinuaga/carlospinuagathevalley-IntroPandas2023/main/products.csv"
url_orders = "https://raw.githubusercontent.com/cpinuaga/carlospinuagathevalley-IntroPandas2023/main/orders.csv"
url_returns = "https://raw.githubusercontent.com/cpinuaga/carlospinuagathevalley-IntroPandas2023/main/returns.csv"

**Ejercicio 1.a**

Lee el archivo de Orders en un DataFrame.  Partelo en dos dataframes de aprox la mitad de tamaño. Comprueba que efectivamente los nuevos dataframes tienen la mitad de filas con la funcion len() y muestralo con la funcion print().

In [ ]:
orders = pd.read_csv(url_orders)

print(len(orders))
orders1 = orders[0:4997]
orders2 = orders[4997:]

print(len(orders1))
print(len(orders2))

In [ ]:
# Podemos ser más exactos de la siguiente manera, evitando tener que conocer la longitud del dataframe con antelación:
orders = pd.read_csv(url_orders)

orders1 = orders[0:len(orders)//2]
orders2 = orders[len(orders)//2:]

# El // se utiliza para realizar una división de números enteros hasta el siguiente entero más pequeño.

print(len(orders1))
print(len(orders2))

**Ejercicio 1.b** Vuelve a juntar los dos dataframes que creaste en el ejercicio anterior, haciendo uso de la función concat()

In [ ]:
orders3 = pd.concat([orders1, orders2])
orders3

**Ejercicio 1.c**


Utilizando la función merge(), une los datos de los pedidos (orders) y clientes (customers) realizando un cruce por la izquierda.

In [ ]:
customers = pd.read_csv(url_customers)

orders.merge(customers, how="left", on="Customer_ID")

**Ejercicio 1.d** Hazlo esta vez utilizando la funcion join(), para lo que tendras que indicar el index.

In [ ]:
customers = customers.set_index("Customer_ID")

orders.join(customers, on="Customer_ID")

**Ejercicio 1.e**
Cambia el nombre de la columna 'Product_ID' de la tabla de órdenes por 'Purchased product ID':

In [ ]:
orders = orders.rename(columns={"Product_ID": "Purchased product ID"})
orders

**Ejercicio 1.f**
Lee los datos de products y crúzalos con los datos de orders indicando la columna por la que cruzar en cada DataFrame. Asegura que solo te quedas con los productos que aparecen en ambos DataFrames haciendo un inner join.

In [ ]:
products = pd.read_csv(url_products)
products.merge(orders, how='inner', left_on='Product_ID', right_on='Purchased product ID')

# Ejercicio 2

En este ejercicio vamos a practicar a trasnformar  nuestro dataframe con las funciones map() apply() y las funciones lambda.

In [ ]:
#Ejecuta este código para trabajar con el DataFrame de Pedidos y Productos
orders = pd.read_csv(url_orders)
products = pd.read_csv(url_products)
orders_and_products = orders.merge(products, how="left", on="Product_ID")
orders_and_products.head()

**Ejercicio 2.a** Crea una función que convierta de Dolares a Euros, utilizando el tipo de cambio 0.93€.

Utilizando map(), crea una nueva columna Price_Eur que convierta Price  (suponiendo que está en dólares) a Euros, con la función que declaraste anteriormente.


Prueba también con apply(), ¿funciona igual?

In [ ]:
def convierte_euros(x):
  return x * 0.93
orders_and_products["Price_Eur"] = orders_and_products["Price"].map(convierte_euros)
orders_and_products.head()

In [ ]:
orders_and_products["Price_Eur"] = orders_and_products["Price"].apply(convierte_euros)
orders_and_products.head()

**Ejercicio 2.b** Crea una nueva columna "Amount" que sea el resultado de multiplicar Price y Quantity utilizando una funcion lambda y apply().

In [ ]:
orders_and_products["Amount"] = orders_and_products.apply(lambda x: x["Price"] * x["Quantity"], axis=1)
orders_and_products.head()

**Ejercicio 2.c** Ahora, aplica la funcion Apply() sobre las filas del dataframe (tendrás que itilizar axis=1 para ello), y crea una lambda  o una función que calcule un descuento utilizando la columna "Amount" del ejercicio anterior.

Si la subcategoría es "Furnishings" o "Labels" que aplique un 12% de descuento, si no, un 7%.

Guarda el resultado en una columna nueva llamada por ejemplo, "Discounted_Amount", pero puedes poner el nombre que quieras.

In [ ]:
orders_and_products["Discounted_Price"] = orders_and_products.apply(lambda x: x["Amount"] * 0.88 if x["Sub_Category"] in ["Furnishings","Labels"] else x["Amount"]*0.93, axis=1)
orders_and_products.head()

In [ ]:
# Alternativa usando una función
def calc_discount(x):
  if x["Sub_Category"] in ["Furnishings","Labels"]:
    return x["Amount"] * 0.88
  else:
    return x["Amount"]*0.93

orders_and_products["Discounted_Price"] = orders_and_products.apply(calc_discount, axis=1)
orders_and_products.head()

**Ejercicio 2.c** Ahora agrupa el dataset por cliente y suma amount para ver cuanto se ha gastado cada uno. Ordena el resultado para saber qué clientes han gastado más.

In [ ]:
orders_and_products.groupby(by="Customer_ID")["Amount"].agg("sum").sort_values(ascending=False)

**Ejercicio 2.d** Esta vez agrupa por cliente y categoría y utiliza la funcion agg para sumar (función sum) la columna amount y contar ids únicos (función nunique()) de productos diferentes. Veremos cuanto ha gastado cada cliente por categoría.

In [ ]:
orders_and_products.groupby(by=["Customer_ID", "Category"]).agg({"Amount": "sum", "Product_ID": "nunique"})


**Ejercicio 2.e**
Crea una columna que categorice los productos según su precio.
* Barato: menor que 300
* Estándar: mayor o igual que 300 y menor que 800
* Premium: mayor o igual que 800

Después encuentra el cliente que mayor número de productos Premium ha comprado, y el cliente que más productos Baratos ha comprado.

In [ ]:
def calc_price_category(x):
  if x['Price'] < 300:
    return 'Barato'
  elif x['Price'] >= 300 and x['Price'] < 800:
    return 'Estándar'
  elif x['Price'] >= 800:
    return 'Premium'

orders_and_products['Price_Category'] = orders_and_products.apply(calc_price_category, axis=1)

orders_and_products[orders_and_products['Price_Category'] == 'Premium'].groupby(by=["Customer_ID"])['Quantity'].agg('sum').sort_values(ascending=False)

In [ ]:
orders_and_products[orders_and_products['Price_Category'] == 'Barato'].groupby(by=["Customer_ID"])['Quantity'].agg('sum').sort_values(ascending=False)